This notebook is dedicated to the sampling of $\boldsymbol{\alpha}$

- The function MCMC_alpha generates MCMC samples $(\boldsymbol{A})_{i=1}^{N}$, taking as argument "df_Lambda" a sample $(\boldsymbol{\lambda}_k)_{k=1}^M$ i.i.d. with density $p_{\boldsymbol{\Lambda}}(.\mid \boldsymbol{\alpha}^{\star})$, "stored_likelihoods" the associated likelihood, "p_lambda_alphastar" the prior densities $p_{\boldsymbol{\Lambda}}(\boldsymbol{\lambda}_k\mid \boldsymbol{\alpha}^\star)_{k=1}^{n}$ computed with $\boldsymbol{\alpha}^\star$, "scale" the standard deviation of the truncated normal prior, "alpha_star" the estimated maximum a posterior for the hyperparameters $\boldsymbol{\alpha}$, "tune_size" the burnin sample size, "size" the sample size, and "rngseed" the random seed
- The function MCMC_alpha_multichains generate multichains MCMC samples with the same arguments as MCMC_alpha
- The function MCMC_alpha_loo uses MCMC_alpha_multichains to generate the MCMC samples for each idx_loo. For a given calibration variable index_calib, it generates $(\boldsymbol{\lambda}_k)_{k=1}^M$ i.i.d. with density $p_{\boldsymbol{\Lambda}}(.\mid \boldsymbol{\alpha}^{\star})$, compute the associated likelihoods and the prior densities $p_{\boldsymbol{\Lambda}}(\boldsymbol{\lambda}_k\mid \boldsymbol{\alpha}^\star)_{k=1}^{n}$,and then generates the MCMC samples $(\boldsymbol{A})_{i=1}^{N}.$

In [ ]:
def MCMC_alpha(df_Lambda, stored_likelihoods, p_lambda_alphastar, scale, alpha_star, tune_size, size, rngseed):

    def ssfun(theta, data): #log likelihood function
        return -2*np.log(max(10**(-30), likelihood_alpha(alpha = theta, likelihoods_alpha_star = stored_likelihoods, denom_is = p_lambda_alphastar, df_Lambda = df_Lambda, index_lambda_p = index_lambda_p, index_lambda_q = index_lambda_q, scale = scale) ))
                         
    bounds = [(max(alpha_star[ii] - delta_alpha,alpha_min), min(alpha_max, alpha_star[ii]+delta_alpha)) for ii in range(len(alpha_star))] #bounds 

    mcstat = MCMC(rngseed=rngseed) 

    x = np.array(range(10))
    y = x.copy()
    mcstat.data.add_data_set(x, y)
    mcstat.simulation_options.define_simulation_options(
        nsimu=int(tune_size+size),
        updatesigma=False, verbosity = 0, waitbar= True)
    mcstat.model_settings.define_model_settings(sos_function=ssfun)
    for ii in range(len(alpha_star)):
        mcstat.parameters.add_model_parameter(
            name=str('$alpha{}$'.format(ii + 1)),
            theta0 = alpha_star[ii],
            minimum = bounds[ii][0],
            maximum = bounds[ii][1]
            )

    mcstat.run_simulation()
    return mcstat.simulation_results.results

def MCMC_alpha_multichains(df_Lambda, stored_likelihoods, p_lambda_alphastar, scale, alpha_star, tune_size, size, rngseed):
    np.random.seed(rngseed)
    seeds = np.random.randint(1000, size = num_chain) #get seed for each chain
    res = [MCMC_alpha(df_Lambda = df_Lambda, stored_likelihoods = stored_likelihoods, p_lambda_alphastar = p_lambda_alphastar, scale = scale, alpha_star = alpha_star, tune_size = tune_size, size = size, rngseed = ss) for ss in seeds] #run every MCMC chain
    samples = np.concatenate([res[i]["chain"][tune_size:,] for i in range(len(res))]) #concatenate the chain without the burnin phase
    return samples

def MCMC_alpha_loo(index_calib, scale, tune_size, size, rngseed):
    alpha_df = pd.read_csv(pre_path + f"/calib_{index_calib}/alpha_df.csv", index_col = 0).values
    for idx_loo in range(len(results_measures)):
        print(idx_loo)
        alpha_star = alpha_df[idx_loo]
        np.random.seed(123456)
        df_Lambda = sample_Lambda(alpha = alpha_star, M = M, index_lambda_p = index_lambda_p, index_lambda_q = index_lambda_q) #sample lambda
        Ysimu_list, Ystd_list, stored_likelihoods = get_likelihoods_dflambda(df_Lambda = df_Lambda.values, sigma = sigma, myCODE = myCODE, results_measures = results_measures, index=[index_calib], std_code = True, idx_loo = idx_loo) #get likelihoods
        p_lambda_alphastar = p_lambda_df(df_Lambda, alpha_star, index_lambda_p, index_lambda_q) #get the prior densities with alpha_star
        samples_alpha_post = MCMC_alpha_multichains(df_Lambda = df_Lambda, stored_likelihoods = stored_likelihoods, p_lambda_alphastar = p_lambda_alphastar, scale = scale, alpha_star = alpha_star, tune_size = tune_size, size = size, rngseed = rngseed) #run multichain MCMC
        save_results(pd.DataFrame(samples_alpha_post), f"samples_alpha_post_{idx_loo}.csv", pre_path = pre_path, calib = index_calib)


In [4]:
plot_bool = False 
tune_size = 1000
size = 250
rngseed = 10
num_chain = 3
M = 2000

alpha_min = -10
alpha_max = 10
delta_alpha = 4


[MCMC_alpha_loo(index_calib = index_calib, scale = scale, tune_size = tune_size, size = size, rngseed = rngseed) for index_calib in calib_only]